In [1]:
import numpy as np
import pandas as pd
import h5py
import os
import gc
import time
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.set_option('display.float_format', lambda x: '%.3f' % x)

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
SWIFT_folder = r'C:\Projects\SWIFT\SWIFT_Workspace\Scenarios'
SCEN = 'Scenario_2045_S12_Beta'
flat_skim_file = os.path.join(SWIFT_folder, SCEN, r'STM\STM_A\01_DynusT\03_Model\Vehicle_Skim.csv')

In [3]:
def read_trajectory_skim_file(flat_skim_file, chunksize=5_000_000):
    
    """
    
    Read the csv file generated from vehicle trajectory itf file
    
    """
    
    df_list = []
    
    dtypes = {
        'vid': np.int32, 
        'tag': np.int8,
        'origz': np.int16,
        'destz': np.int16,
        'class': np.uint8,
        'stime': np.float16,
        'travel_time': np.float16,
        'nodes': np.int32,
        'vehtype': np.uint8,
        'vot': np.float16,
        'toll': np.float16,
    }
    
    usecols = tuple(dtypes.keys())
    
    #  
    for df_chunk in tqdm(pd.read_csv(flat_skim_file, chunksize=chunksize, usecols=usecols, dtype=dtypes)):
    
#         expand_data(df_chunk)
        df_list.append(df_chunk) 
    
    df = pd.concat(df_list)
    del df_list
    return df

In [18]:
def read_vehicle_skim_file(flat_skim_file, chunksize=5_000_000):
    """
    
    Read the csv file generated from b03 trajectories
    
    """
    df_list = []
    
    dtypes = {
        'vid': np.int32, 
        'tag': np.int8,
        'uclass': np.int8,
        'origz': np.int16,
        'destz': np.int16,
        'start': np.float16,
        'end': np.float16,
        'delay': np.float16,
        'dist': np.float16,
        'nodes': np.int32,
        'node0': np.int32,
        'node1': np.int32,
        'vtyp': np.uint8,
        'totveh': np.int64,
        'purpose': np.uint8,
    }
    
    usecols = tuple(dtypes.keys())
    
    for df_chunk in tqdm(pd.read_csv(flat_skim_file, chunksize=chunksize, usecols=usecols, dtype=dtypes)):
        df_list.append(df_chunk) 
    
    df = pd.concat(df_list)
    del df_list
    return df

In [19]:
df = read_vehicle_skim_file(flat_skim_file)

A Jupyter Widget

In [7]:
df.columns

Index(['vid', 'tag', 'uclass', 'vtyp', 'origz', 'destz', 'nodes', 'node0',
       'node1', 'start', 'end', 'delay', 'dist', 'totveh', 'purpose'],
      dtype='object')

In [8]:
df.shape

(30804331, 15)

In [21]:
df.tail()

,vid,tag,uclass,vtyp,origz,destz,nodes,node0,node1,start,end,delay,dist,totveh,purpose
30804326,30804166,1,3,5,3662,3668,1,390038,364069,1438.000,1620.000,0.600,0.357,3568392492,20
30804327,30804184,1,3,6,3668,3662,2,390038,369558,1438.000,1620.000,0.700,5.742,3568392496,2
30804328,30804205,1,3,1,3932,4072,27,396120,362414,1438.000,1620.000,0.525,20.734,3568392504,20
30804329,30804240,1,3,1,4216,1104,7,430382,426224,1438.000,1620.000,0.000,2.906,3568392612,10
30804330,30804278,1,3,1,3669,3693,1,390038,364069,1439.000,1620.000,0.000,0.396,3568392640,20


In [23]:
df['travel_time'] = df.end - df.start

In [16]:
# which completed (tag == 2) trip has the longest travel time
df.loc[df.loc[df.tag == 2].travel_time.idxmax()]

vid              9654552.000
tag                    2.000
uclass                 3.000
vtyp                   1.000
origz               3668.000
destz               3662.000
nodes                 14.000
node0             390038.000
node1             364069.000
start                661.000
end                 1591.000
delay                661.500
dist                  27.578
totveh        -729470076.000
purpose               10.000
travel_time          930.000
Name: 30685567, dtype: float64

In [24]:
# for each vtyp, which trip has the longest travel time; Split-apply-combine paradigm
def longest_travel_time(g):
    return g.loc[g.loc[g.tag == 2].travel_time.idxmax()]

df.groupby('vtyp').apply(longest_travel_time)


,vid,tag,uclass,vtyp,origz,destz,nodes,node0,node1,start,end,delay,dist,totveh,purpose,travel_time
vtyp,,,,,,,,,,,,,,,,
1,9654552.000,2.000,3.000,1.000,3668.000,3662.000,14.000,390038.000,364069.000,661.000,1591.000,661.500,27.578,3565497220.000,10.000,930.000
2,10876293.000,2.000,3.000,2.000,3668.000,5252.000,39.000,390038.000,364069.000,702.000,1620.000,23.125,69.125,3566151592.000,3.000,918.000
3,10978697.000,2.000,3.000,3.000,3669.000,3534.000,53.000,390038.000,364069.000,705.500,1601.000,23.953,37.438,3565734012.000,50.000,895.500
4,12282648.000,2.000,3.000,4.000,3871.000,5261.000,40.000,398165.000,397516.000,750.500,1573.000,796.000,17.281,3565040400.000,6.000,822.500
5,9701882.000,2.000,3.000,5.000,3662.000,3662.000,14.000,390038.000,364069.000,662.500,1591.000,662.500,27.047,3565502424.000,20.000,928.500
6,8973484.000,2.000,3.000,6.000,3668.000,3672.000,11.000,390038.000,364069.000,637.000,1564.000,656.500,23.016,3564747060.000,10.000,927.000
9,11168666.000,2.000,3.000,9.000,3669.000,3668.000,10.000,390038.000,364069.000,712.000,1589.000,19.500,22.672,3565451572.000,40.000,877.000


In [22]:
np.sum(df.travel_time.values, dtype=np.float64) / 60.0

18749044.308287453

In [17]:
df.loc[df.vid == 24476]

,vid,tag,origz,destz,class,stime,travel_time,nodes,vehtype,vot,toll
18150,24476,2,909,915,3,35.094,4.648,11,2,30.797,0.000


In [18]:
df.dtypes

vid              int32
tag               int8
origz            int16
destz            int16
class            uint8
stime          float16
travel_time    float16
nodes            int32
vehtype          uint8
vot            float16
toll           float16
dtype: object

In [25]:
transims_skim_file = os.path.join(SWIFT_folder, SCEN, r'STM\STM_A\02_TrafficPredictor\03_Demand\FINAL_Skims.csv')

In [27]:
df_transims_skim = pd.read_csv(transims_skim_file)

KeyboardInterrupt: 

In [ ]:
df_transims_skim.columns

In [ ]:
df_transims_skim.columns.loc[df_transims_skim.columns.HHOLD == 24476]

In [ ]:
df_transims_tt = df_transims_skim.loc[:, ['HHOLD', 'TIME']]

In [ ]:
df_join = pd.merge(df, df_transims_tt, left_on='vid', right_on='HHOLD')

In [ ]:
df_join['TIME'] = df_join['TIME'] / 60.0

In [ ]:
df_join['diff'] = df_join.travel-time = df_join.TIME

In [ ]:
df_join.diff.describe()